In [5]:
!pip install scikit-surprise
!pip install sqlalchemy
!pip install pymysql
!pip install caserecommender

In [14]:
import pandas as pd
# from surprise import Dataset, Reader
# from surprise.model_selection import train_test_split
# from surprise import SVD  # or any other algorithm you want to use
# from surprise import accuracy
from caserec.recommenders.item_recommendation.userknn import UserKNN
from caserec.utils.cross_validation import CrossValidation
from caserec.utils.process_data import ReadFile
import urllib.parse
# Load your dataset
from sqlalchemy import create_engine
from sqlalchemy import text

# Load your dataset
params = urllib.parse.quote_plus(r'Driver={ODBC Driver 18 for SQL Server};Server=tcp:adaptive-learning-server.database.windows.net,1433;Database=adaptive_learning_db;Uid=superadmin;Pwd=Poorpassword@2024;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;')
conn_str = 'mssql+pyodbc:///?odbc_connect={}'.format(params)
engine_azure = create_engine(conn_str,echo=True)

connection = engine_azure.connect()
print("Connection successful !!!!")

# Perform a query to fetch effectivness
query = text(
    "SELECT * "
    "FROM rating_materials;"
)

result = connection.execute(query)

# Fetch all the rows and convert them to a Pandas DataFrame
df = pd.DataFrame(result.fetchall(), columns=result.keys())
# print("!!! df retrieved values:", df)

file_path = '../LA--AdaptiveLearning/u.data'

# Convert the DataFrame to the u.data format
df[['user_id', 'item_id', 'rating']].to_csv(file_path, sep='\t', index=False, header=False)

# Use CaseRecommender's ReadFile to read the data
# data_reader = ReadFile(input_file=file_path, separator='\t')
# data_reader = data_reader.read()

recommender = UserKNN()
CrossValidation(input_file=file_path, recommender=recommender, dir_folds='../LA--AdaptiveLearning/', header=1, k_folds=5, write_predictions='TRUE').compute()


2024-01-30 22:08:26,125 INFO sqlalchemy.engine.Engine SELECT CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR)
2024-01-30 22:08:26,126 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-30 22:08:26,156 INFO sqlalchemy.engine.Engine SELECT schema_name()
2024-01-30 22:08:26,157 INFO sqlalchemy.engine.Engine [generated in 0.00068s] ()
2024-01-30 22:08:26,226 INFO sqlalchemy.engine.Engine SELECT CAST('test max support' AS NVARCHAR(max))
2024-01-30 22:08:26,227 INFO sqlalchemy.engine.Engine [generated in 0.00134s] ()
2024-01-30 22:08:26,248 INFO sqlalchemy.engine.Engine SELECT 1 FROM fn_listextendedproperty(default, default, default, default, default, default, default)
2024-01-30 22:08:26,249 INFO sqlalchemy.engine.Engine [generated in 0.00150s] ()
Connection successful !!!!
2024-01-30 22:08:26,293 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-30 22:08:26,294 INFO sqlalchemy.engine.Engine SELECT * FROM rating_materials;
2024-01-30 22:08:26,294 INFO sqlalchemy.engine.Engine [generat

In [21]:
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy

# Define the rating scale based on your data
reader = Reader(rating_scale=(1, 6))  # Adjust min_rating and max_rating as per your data
data = Dataset.load_from_df(df[['user_id', 'item_id', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.25)  # Adjust test_size as per your preference




RMSE: 1.5986


1.5986261435477627

In [31]:
import pandas as pd
#from caserec.recommenders.hybrid import MMR
from caserec.recommenders.item_recommendation.userknn import UserKNN
from caserec.recommenders.item_recommendation.itemknn import ItemKNN
import urllib.parse
# Load your dataset
from sqlalchemy import create_engine
from sqlalchemy import text

# Load your dataset
params = urllib.parse.quote_plus(r'Driver={ODBC Driver 18 for SQL Server};Server=tcp:adaptive-learning-server.database.windows.net,1433;Database=adaptive_learning_db;Uid=superadmin;Pwd=Poorpassword@2024;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;')
conn_str = 'mssql+pyodbc:///?odbc_connect={}'.format(params)
engine_azure = create_engine(conn_str,echo=True)

connection = engine_azure.connect()
print("Connection successful !!!!")

# Perform a query to fetch effectivness
query_uim = text(
    "SELECT * "
    "FROM rating_materials;"
)

result_uim = connection.execute(query_uim)

# Fetch all the rows and convert them to a Pandas DataFrame
df_uim = pd.DataFrame(result_uim.fetchall(), columns=result_uim.keys())
print("df_uim : !!!!! ", df_uim)
# print("!!! df retrieved values:", df)

# file_path = '../LA--AdaptiveLearning/u.data'
# Perform a query to fetch effectivness
query_dd = text(
    "SELECT * "
    "FROM user_input_data_integer;"
)

result_dd = connection.execute(query_dd)

# Fetch all the rows and convert them to a Pandas DataFrame
df_dd = pd.DataFrame(result_dd.fetchall(), columns=result_dd.keys())
print("df_dd : !!!!! ", df_dd)
# # Convert the DataFrame to the u.data format
# df[['user_id', 'item_id', 'rating']].to_csv(file_path, sep='\t', index=False, header=False)
# Assuming df is a DataFrame with columns: user_id, item_id, rating, age, gender, study_field, etc.

# Step 1: Create User-Item Matrix
user_item_matrix = df_uim.pivot_table(index='user_id', columns='item_id', values='rating', fill_value=0)

# Step 2: Apply Clustering Algorithm (e.g., K-means) on Demographic Data
demographic_data = df_dd[['New_user_id', 'Age', 'Gender', 'Study_Field', 'Educational_Level', 'Employment_Status',
                          'Preferred_Study_Type', 'Study_Place', 'Study_Sound', 'Studying_Time', 'Studying_Classes',
                          'Support_by_Study_Difficulty', 'Likert_Platforms_Effectiveness', 'Trustworthy_of_AI',
                          'Relying_on_Recommendation_Paths']].drop_duplicates().set_index('New_user_id')
# ... preprocess demographic_data if needed ...

# ... preprocess demographic_data if needed ...

# Assuming 'user_id' is used as an index in both user_item_matrix and demographic_data
# Combine user-item interactions with demographic data
user_data_matrix = pd.concat([user_item_matrix, demographic_data], axis=1)

# Apply clustering to demographic data
from sklearn.cluster import KMeans

num_clusters = 5  # Adjust the number of clusters as needed
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
user_clusters = kmeans.fit_predict(demographic_data)

# Step 3: Assign a New User to a Cluster (Simulate with a new user's preferences and demographic data)
# new_user_preferences = [4, 5, 0, 0, 0, 0]  # Ratings for items, where 0 indicates no interaction
new_user_demographics = {
    'Age': 25,
    'Gender': '1',
    'Study_Field': '1',
    'Educational_Level': '1',
    'Employment_Status': '1',
    'Preferred_Study_Type': '1',
    'Study_Place': '1',
    'Study_Sound': '1',
    'Studying_Time': '1',
    'Studying_Classes': '1',
    'Support_by_Study_Difficulty': '1',
    'Likert_Platforms_Effectiveness': '1',
    'Trustworthy_of_AI': '1',
    'Relying_on_Recommendation_Paths': '1'
}
# Assign the new user to a demographic cluster
new_user_cluster = kmeans.predict([list(new_user_demographics.values())])[0]
print("Cluster NEW USER!!!" , new_user_cluster)

# Step 4: Get Similar Users from the Same Cluster
cluster_members = demographic_data.index[user_clusters == new_user_cluster]
print("Cluster Members!!!" , cluster_members)

# Step 5: Prepare Input for CaseRecommender
train_file = 'train_file.dat'  # File to store training data in CaseRecommender format
with open(train_file, 'w') as file:
    for user in cluster_members:
        items_rated = df[df['user_id'] == user][['item_id', 'rating']].values
        for item, rating in items_rated:
            file.write(f"{user}::{item}::{rating}\n")

# Step 6: Train a Recommendation Model (e.g., UserKNN)
train = UserKNN(train_file, similarity_metric='cosine')
train.compute()

# Step 7: Make Recommendations for the New User using MMR (Maximal Marginal Relevance)
# hybrid_recommender = MMR(train_file, train_file, n_recommendations=5, diversity_factor=0.5)
# hybrid_recommender.compute()

# recommendations = hybrid_recommender.recommend(new_user_preferences, new_user_demographics)

# print("Recommendations for the new user:")
# print(recommendations)

2024-01-30 23:04:51,638 INFO sqlalchemy.engine.Engine SELECT CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR)
2024-01-30 23:04:51,639 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-30 23:04:51,664 INFO sqlalchemy.engine.Engine SELECT schema_name()
2024-01-30 23:04:51,664 INFO sqlalchemy.engine.Engine [generated in 0.00053s] ()
2024-01-30 23:04:51,731 INFO sqlalchemy.engine.Engine SELECT CAST('test max support' AS NVARCHAR(max))
2024-01-30 23:04:51,732 INFO sqlalchemy.engine.Engine [generated in 0.00051s] ()
2024-01-30 23:04:51,756 INFO sqlalchemy.engine.Engine SELECT 1 FROM fn_listextendedproperty(default, default, default, default, default, default, default)
2024-01-30 23:04:51,757 INFO sqlalchemy.engine.Engine [generated in 0.00060s] ()
Connection successful !!!!
2024-01-30 23:04:51,801 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-30 23:04:51,801 INFO sqlalchemy.engine.Engine SELECT * FROM rating_materials;
2024-01-30 23:04:51,801 INFO sqlalchemy.engine.Engine [generat

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(


ZeroDivisionError: float division by zero

In [22]:
from surprise import SVD  # or any other algorithm you want to use

# Train your model
algo = SVD()  # or any other algorithm
algo.fit(trainset)

# Make predictions and evaluate the model
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 1.5874


1.5874238136082583

In [27]:
from surprise import prediction_algorithms  # or any other algorithm you want to use

# Train your model
algo = prediction_algorithms.KNNBasic()  # or any other algorithm
algo.fit(trainset)

# Make predictions and evaluate the model
predictions = algo.test(testset)
accuracy.rmse(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.4508


1.4507999945135885

In [29]:
from surprise import prediction_algorithms  # or any other algorithm you want to use

# Train your model
algo = prediction_algorithms.BaselineOnly()  # or any other algorithm
algo.fit(trainset)

# Make predictions and evaluate the model
predictions = algo.test(testset)
accuracy.rmse(predictions)

Estimating biases using als...
RMSE: 1.6299


1.6299402661337103

In [30]:
from surprise import prediction_algorithms  # or any other algorithm you want to use

# Train your model
algo = prediction_algorithms.KNNBaseline()  # or any other algorithm
algo.fit(trainset)

# Make predictions and evaluate the model
predictions = algo.test(testset)
accuracy.rmse(predictions)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.4868


1.4868398260101279

In [31]:
from surprise import prediction_algorithms  # or any other algorithm you want to use

# Train your model
algo = prediction_algorithms.KNNWithMeans()  # or any other algorithm
algo.fit(trainset)

# Make predictions and evaluate the model
predictions = algo.test(testset)
accuracy.rmse(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.6856


1.6856106484719349

In [33]:
from surprise import prediction_algorithms  # or any other algorithm you want to use

# Train your model
algo = prediction_algorithms.NormalPredictor()  # or any other algorithm
algo.fit(trainset)

# Make predictions and evaluate the model
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 2.2157


2.215749310031422

In [34]:
from surprise import prediction_algorithms  # or any other algorithm you want to use

# Train your model
algo = prediction_algorithms.KNNWithZScore()  # or any other algorithm
algo.fit(trainset)

# Make predictions and evaluate the model
predictions = algo.test(testset)
accuracy.rmse(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.7316


1.7316291982916088